## BFGS Algorithm

In [1]:
function rosenbrock_BFGS( Θ, ITERS )
    
    DIM = length(Θ)
    Q = Matrix(1.0I, DIM, DIM)
    arguments = zeros(DIM, ITERS)
    values = []
    
    for i in 1:ITERS
        
        # History
        arguments[1,i] = round( Θ[1], digits=8 )
        arguments[2,i] = round( Θ[2], digits=8 )
        push!(values, round(rosenbrock(Θ), digits=8))
        
        # Finding new weights and update
        Θ, Q = BFGS( Θ, Q )
    
    end
    return arguments, values, Q
end;

In [2]:
function BFGS( Θ, Q ) # author: Bartosz Chaber
    
    f = rosenbrock
    ∇f = rosenbrock_gradient
    g = ∇f(Θ)
    
    # Looking for the best step size
    d  = -Q*g    # Direction
    ϕ  = α -> f(Θ + α*d)
    ϕ′ = α -> ∇f(Θ + α*d) ⋅ d
    α = line_search( ϕ, ϕ′, d )  #  α is not a hyperparameter!
    
    # New weights
    Θ′ = Θ + α * d
    
    # Hessian Approximation Update
    g′ = ∇f( Θ′ )
    δ = Θ′ - Θ
    γ = g′ - g    
    Q′ = Q - (δ*γ'*Q + Q*γ*δ') / (δ'*γ) +
                 (1.0 + (γ'*Q*γ) / (δ'*γ)) *
                          (δ*δ') / (δ'*γ)
    
    return Θ′, Q′

end;

In [3]:
function line_search( ϕ, ϕ′, d ) # author: Bartosz Chaber
    a, b = bracket_minimum(ϕ)
    x, y = golden_section_search(ϕ, a, b)
    x/2 + y/2
end;

In [4]:
function bracket_minimum(f; x=0, s=1e-2, k=2.0) # author: the book
    a, ya = x, f(x)
    b, yb = a + s, f(a + s)
    if yb > ya
        a, b = b, a
        ya, yb = yb, ya
        s = -s
    end
    while true
        c, yc = b + s, f(b + s)
        if yc > yb
            return a < c ? (a, c) : (c, a)
        end
        a, ya, b, yb = b, yb, c, yc
        s *= k
    end
end;

In [5]:
import Base.MathConstants: φ

function golden_section_search( f, a, b; n=50 ) # author: the book
    ρ = φ-1
    d = ρ * b + (1 - ρ)*a
    yd = f(d)
    for i = 1 : n-1
        c = ρ*a + (1 - ρ)*b
        yc = f(c)
        if yc < yd
            b, d, yd = d, c, yc
        else
            a, b = b, c
        end
    end
    return a < b ? (a, b) : (b, a)
end;

## L-BFGS Algorithm

In [6]:
function rosenbrock_LBFGS( Θ, ITERS, m )
    DIM = length(Θ)
    δs, γs, qs = [], [], []
    arguments = zeros(DIM, ITERS)
    values = []
    
    for i in 1:ITERS
        # History
        arguments[1,i] = round( Θ[1], digits=8 )
        arguments[2,i] = round( Θ[2], digits=8 )
        push!(values, round(rosenbrock(Θ), digits=8))
        
        # Finding new weights and update
        Θ, δs, γs, qs = LBFGS(Θ, m, δs, γs, qs)    
    end
    return arguments, values, δs, γs, qs
end;

rosenbrock_LBFGS (generic function with 1 method)

In [7]:
function LBFGS(θ, m, δs, γs, qs) # author: Bartosz Chaber
    f = rosenbrock
    ∇f = rosenbrock_gradient

    n, g = length(δs), ∇f(θ)
    d = -g
    if n > 0
        q=g
        for i in n:-1:1
            qs[i] = copy(q)
            q -= (δs[i]⋅q) / (γs[i]⋅δs[i]) * γs[i]
        end
        z = (γs[n] .* δs[n] .* q) / (γs[n]⋅γs[n])
        for i in 1:+1:n
            z += δs[i]*(δs[i]⋅qs[i]-γs[i]⋅z)/(γs[i]⋅δs[i])
        end
        d = -z;
    end
    φ =α-> f(θ+α*d); φ′=α->∇f(θ+α*d)⋅d
    α = line_search(φ, φ′, d)
    θ′ = θ + α*d; g′ = ∇f(θ′)
    δ =θ′-θ;γ =g′-g
    push!(δs, δ); push!(γs, γ); push!(qs, zero(θ))
    while length(δs) > m
        popfirst!(δs); popfirst!(γs); popfirst!(qs)
    end
    return θ′, δs, γs, qs
end;

LBFGS (generic function with 1 method)